In [1]:
# mount Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q -U peft transformers datasets bitsandbytes trl==0.19.1 accelerate wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Prep dataset

In [3]:
# prepare data
from datasets import load_dataset
eval_dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem", split="train[:1%]")

README.md: 0.00B [00:00, ?B/s]

medical_o1_verifiable_problem.json:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/40644 [00:00<?, ? examples/s]

In [4]:
eval_dataset

Dataset({
    features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
    num_rows: 406
})

In [5]:
import pandas as pd
eval_dataset = pd.DataFrame(eval_dataset)
eval_dataset.head()

,Open-ended Verifiable Question,Ground-True Answer
0,An 88-year-old woman with osteoarthritis is ex...,Gastric ulcer
1,In the context of disseminated intravascular c...,Fibrin degradation products
2,"In a 3-year-old boy with severe diarrhea, vomi...","Double-stranded, icosahedral, non-enveloped"
3,Based on the chest radiograph and abdominal CT...,Hydatid Cyst
4,What is one potential side effect that is not ...,Anaphylaxis


## Set up model definition

In [6]:
# load model
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import userdata

# Load the LoRA model
model_id = "erica-landreth/Meta-Llama-3-8B-Instruct-qlora-medical-r32-limited"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

In [7]:
# Load tokenizer
base_model_id = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True,
                                          token=userdata.get('HF_TOKEN'))
tokenizer.add_special_tokens({'pad_token': '<PAD>'})

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

1

In [8]:
tokenizer.get_chat_template()

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

## Eval of LoRA SFT (R = 32) model

In [9]:
# set up inference pipeline
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model_id,
                model_kwargs={
                    "quantization_config":quantization_config,
                },
                token=userdata.get('notebook2'))


adapter_config.json:   0%|          | 0.00/868 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/109M [00:00<?, ?B/s]

Device set to use cuda:0


In [10]:
fn_out = '/content/drive/MyDrive/w266_project/lora-sft-r32-limited-output-reformatted-reference.txt'
with open(fn_out, 'w') as f:
    f.write('\n')

In [11]:
for idx,sample in enumerate(eval_dataset['Open-ended Verifiable Question']):

  prompt = [
      {"role":"system","content":"You are a medical expert"},
      {"role":"user","content": "Provide a succinct answer to the following question: "+sample}
  ]

  outputs = pipe(
      prompt,
      max_new_tokens=1024,
      temperature=0.01,
      return_full_text=False
  )
  response = outputs[0]["generated_text"]
  response = response.replace('\n','\t')
  print(response)
  with open(fn_out, 'a') as f:
      f.write(f"{idx}*****!!!!!"+response+'\n')

  if idx > 100:
    break

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The most likely cause of the gastrointestinal blood loss in this 88-year-old woman is a gastric ulcer, likely due to the use of naproxen. Naproxen, a nonsteroidal anti-inflammatory drug (NSAID), is known to increase the risk of gastric ulcers, which can lead to bleeding. This is particularly concerning in older adults who may be more susceptible to gastrointestinal complications due to their age and potential comorbidities.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In the context of disseminated intravascular coagulation (DIC), the blood component expected to show an increase due to the excessive breakdown of fibrin is fibrinogen. This occurs because the body attempts to compensate for the rapid consumption of fibrinogen by producing more, leading to an elevated level.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

The primary mechanism of action of the analgesic drug likely prescribed to this patient is the activation of opioid receptors. This is consistent with the patient's history of breast cancer and her use of intravenous heroin, which suggests a need for opioid-based pain management.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


The appropriate surgical management for this patient, given the diagnosis of papillary thyroid carcinoma with no lymph node involvement, is a total thyroidectomy. This approach ensures that all thyroid tissue is removed, thereby preventing any potential recurrence.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Based on the clinical findings and laboratory results, a stone is most likely to be found in the common bile duct on ultrasonography. This conclusion is drawn from the presence of scleral icterus, which suggests a bile duct obstruction, and the elevated levels of bilirubin, alkaline phosphatase, and γ-glutamyltransferase, which are indicative of a biliary obstruction. The colicky abdominal pain and nausea further support this diagnosis.
The most likely diagnosis for the 23-year-old male experiencing recurrent scaly lesions on the glans penis is balanitis circinata. This condition is characterized by the formation of circular, ring-like lesions on the glans penis, which align with the patient's description of the lesions appearing at the same location and healing with slight hyperpigmentation.
